In [1]:
import tensorflow as tf
import os
import pickle
import numpy as np
import CifarData

CIFAR_DIR = "./../cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [2]:
"""
#The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, 
with 6000 images per class. There are 50000 training images and 10000 test images. 

## data -- a 10000x3072(32*32*3) numpy array of uint8s. 
Each row of the array stores a 32x32 colour image. 
The first 1024 entries contain the red channel values, 
the next 1024 the green, and the final 1024 the blue. 
The image is stored in row-major order, so that the first 32 entries of the array 
are the red channel values of the first row of the image.

## labels -- a list of 10000 numbers in the range 0-9. 
The number at index i indicates the label of the ith image in the array data.
"""

'\n#The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, \nwith 6000 images per class. There are 50000 training images and 10000 test images. \n\n## data -- a 10000x3072(32*32*3) numpy array of uint8s. \nEach row of the array stores a 32x32 colour image. \nThe first 1024 entries contain the red channel values, \nthe next 1024 the green, and the final 1024 the blue. \nThe image is stored in row-major order, so that the first 32 entries of the array \nare the red channel values of the first row of the image.\n\n## labels -- a list of 10000 numbers in the range 0-9. \nThe number at index i indicates the label of the ith image in the array data.\n'

In [3]:
"""
testing 
"""
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData.CifarData(train_filenames, True)
test_data = CifarData.CifarData(test_filenames, False)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [4]:
"""
construct the graph
"""
x = tf.placeholder(tf.float32, [None, 3072])
# [None], eg: [0,5,6,3]
y = tf.placeholder(tf.int64, [None])

# three hidden layers, fully connected

hidden1 = tf.layers.dense(x, 100, activation=tf.nn.relu)
hidden2 = tf.layers.dense(hidden1, 100, activation=tf.nn.relu)
hidden3 = tf.layers.dense(hidden2, 50, activation=tf.nn.relu)
y_ = tf.layers.dense(hidden3, 10)

loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# y_ -> sofmax
# y -> one_hot
# loss = ylogy_

# indices
predict = tf.argmax(y_, 1)
# [1,0,1,1,1,0,0,0]
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [5]:
"""
run the graph
"""
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps = 100

# train: 100k: 51.%
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
                                        [loss, accuracy, train_op],
                                        feed_dict={
                                            x: batch_data,
                                            y: batch_labels}
                                        )
        if (i+1) % 500 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' 
                  % (i+1, loss_val, acc_val))
        if (i+1) % 5000 == 0:
            test_data = CifarData.CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                                        [accuracy],
                                        feed_dict = {
                                            x: test_batch_data, 
                                            y: test_batch_labels}
                                        )
                all_test_acc_val.append(test_acc_val)
                
            test_acc = np.mean(all_test_acc_val)
            print('[Test ] Step: %d, acc: %4.5f'% (i+1, test_acc))

[Train] Step: 500, loss: 1.72040, acc: 0.25000
[Train] Step: 1000, loss: 1.60773, acc: 0.35000
[Train] Step: 1500, loss: 1.78879, acc: 0.40000
[Train] Step: 2000, loss: 1.50166, acc: 0.50000
[Train] Step: 2500, loss: 1.97384, acc: 0.35000
[Train] Step: 3000, loss: 1.64070, acc: 0.35000
[Train] Step: 3500, loss: 1.55406, acc: 0.30000
[Train] Step: 4000, loss: 1.14214, acc: 0.75000
[Train] Step: 4500, loss: 1.20850, acc: 0.60000
[Train] Step: 5000, loss: 1.61235, acc: 0.50000
(10000, 3072)
(10000,)
[Test ] Step: 5000, acc: 0.46100
[Train] Step: 5500, loss: 1.42832, acc: 0.40000
[Train] Step: 6000, loss: 1.41347, acc: 0.50000
[Train] Step: 6500, loss: 1.47176, acc: 0.50000
[Train] Step: 7000, loss: 1.46576, acc: 0.55000
[Train] Step: 7500, loss: 1.26129, acc: 0.50000
[Train] Step: 8000, loss: 1.20261, acc: 0.50000
[Train] Step: 8500, loss: 1.45637, acc: 0.50000
[Train] Step: 9000, loss: 1.57878, acc: 0.45000
[Train] Step: 9500, loss: 1.88338, acc: 0.55000
[Train] Step: 10000, loss: 1.4015